# Clonando o repo e instalando a lib

In [1]:
!git clone https://github.com/GCrispino/vi-pddlgym.git
!pip install pddlgym
!cd vi-pddlgym/

fatal: destination path 'vi-pddlgym' already exists and is not an empty directory.


# Definição do domínio e problema

In [6]:
nome = 'batalha' #@param {type: "string"}

acoes = 'socar(personagem_personagem1, personagem_personagem2); socarDeNovo(personagem_personagem1, personagem_personagem2); castarMagia(personagem_personagem1, personagem_personagem2, magia_magia1); prepararMagia(personagem_personagem1, magia_magia1); nocautear(personagem_personagem1, personagem_personagem2)' #@param {type: "string"}
precondicoes = 'vivo(personagem_personagem1); vivo(personagem_personagem1) and ferido(personagem_personagem2); vivo(personagem_personagem1) and preparada(magia_magia1) and pertence(magia_magia1, personagem_personagem1); vivo(personagem_personagem1) and pertence(magia_magia1, personagem_personagem1); vivo(personagem_personagem1) and muitoFerido(personagem_personagem2)' #@param {type: "string"}
efeitos = 'ferido(personagem_personagem2); muitoFerido(personagem_personagem2); muitoFerido(personagem_personagem2) and not_preparada(magia_magia1); preparada(magia_magia1); not_vivo(personagem_personagem2)' #@param {type: "string"}
estado_inicial = 'vivo(personagem_heroi) and not_preparada(magia_magiaHeroi) and pertence(magia_magiaHeroi, personagem_heroi) and vivo(personagem_vilao) and not_preparada(magia_magiaVilao) and pertence(magia_magiaVilao, personagem_vilao)' #@param {type: "string"}
estado_meta = 'not_vivo(personagem_vilao)' #@param {type: "string"}
acoes_problema = 'socar(personagem_heroi, personagem_vilao); socar(personagem_vilao, personagem_heroi); socarDeNovo(personagem_heroi, personagem_vilao); socarDeNovo(personagem_vilao, personagem_heroi); castarMagia(personagem_heroi, personagem_vilao, magia_magiaHeroi); castarMagia(personagem_vilao, personagem_heroi, magia_magiaVilao); prepararMagia(personagem_heroi, magia_magiaHeroi); prepararMagia(personagem_vilao, magia_magiaVilao); nocautear(personagem_heroi, personagem_vilao); nocautear(personagem_vilao, personagem_heroi) ' #@param {type: "string"}

print_domain = False #@param {type: "boolean"}
print_problem = False #@param {type: "boolean"}

acoes = acoes.split('; ') # cook(dinner_dinner, hands_hands); wrap(present_present, sound_sound); carry(garbage_garbage, hands_hands); dolly(garbage_garbage, sound_sound)
preconds = precondicoes.split('; ') # isclean(hands_hands); isquiet(sound_sound); hasgarbage(garbage_garbage); hasgarbage(garbage_garbage)
efeitos = efeitos.split('; ') # isready(dinner_dinner); iswrapped(present_present); not_hasgarbage(garbage_garbage) and not_isclean(hands_hands); not_hasgarbage(garbage_garbage) and not_isquiet(sound_sound)

est_inicial = estado_inicial.split(' and ') # hasgarbage(garbage_g) and isclean(hands_h) and isquiet(sound_s)
est_meta = estado_meta.split(' and ') # isready(dinner_d) and iswrapped(present_p) and not_hasgarbage(garbage_g)
acoes_prob = acoes_problema.split('; ') # cook(dinner_d, hands_h); wrap(present_p, sound_s); carry(garbage_g, hands_h); dolly(garbage_g, sound_s)

# Definicao do dominio
import re
literais = ' '.join(acoes + preconds + efeitos)
types_raw = [i.split(', ') for i in re.findall('(?<=\()[^\)]+(?=\))', literais)]
types_set = set()
for l in types_raw: 
  for l1 in l: types_set.add(l1.split('_')[0])
types = '(:types'
for i in types_set: types += ' '+i
types += ')'

predicados = '(:predicates \n\t'+'\n\t'.join(set(('('+re.sub('\)\s+', ')\n(', re.sub('\(|, ', ' ?', re.sub(' and|not_', '', literais)))).split('\n')))+'\n)'
predicados_final = ''
for i in predicados.split('?'):
  if len(i.split('_')) > 1:
    type_, post = i.split('_')
    name_ = re.findall('\w+', post)[0]
    predicados_final += '?'+(type_+'_'+name_+' - '+type_)+post.split(name_)[1]
  else: predicados_final += i

actions = ';  (:actions' 
action_desc = ''
for a in acoes:
  actions += ' '+a.split('(')[0]
  action_desc += '\n(:action '+a.split('(')[0] + '\n :parameters ( ' 
  parameters = ''
  parameters_literal = ''
  for parameter in (a[:-1].split('(')[1]).split(','): 
    parameters += '?'+parameter.split()[0]+ ' - ' + parameter.split('_')[0]+' '
    parameters_literal += '?'+parameter.split()[0]+' '
  action_desc += parameters + ')\n  :precondition ('
  pre = preconds[acoes.index(a)].split(' and ')
  pre_final = ''
  for p in pre:
    if 'not_' in p:
      pre_final += ' (not (' + re.sub('\(|, ', ' ?', p.split('not_')[1]) + ')'
    else: pre_final += '('+re.sub('\(|, ', ' ?', p) + '\n\t\t'
  action_desc += 'and (' + a.split('(')[0] + ' ' + parameters_literal + ')\n\t\t' + pre_final + '\n\t\t'
  action_desc += ')\n  :effect ('
  ef = efeitos[acoes.index(a)].split(' and ')
  ef_final = ''
  for p in ef:
    if 'not_' in p:
      ef_final += ' (not (' + re.sub('\(', ' ?', p.split('not_')[1]) + ')'
    else: ef_final += '(' + re.sub('\(', ' ?', p) + '\n\t\t'
  action_desc += 'and ' + ef_final + '\n\t\t'
  action_desc += ')\n)\n'
actions += ')\n'

pddl = '(define (domain '+nome+''')
        (:requirements :typing)\n\t'''+types + '\n' + predicados_final + '\n\n' + actions + action_desc + '\n)'

dominio = re.sub('(?<=\()\s|\s(?=\))|(?<=-\s)\s', '', pddl)+')'

if print_domain: print(dominio)




# Definicao do problema
literals = ' '.join(est_inicial + est_meta + acoes_prob)

objs = set(re.findall('\w+_\w+', literals))
objs = {o for o in objs if 'not_' not in o}
obj_str = '(:objects\n'
for o in objs: obj_str += '\t' + o + ' - ' + o.split('_')[0] + '\n'
obj_str += ')'

init = '(:init'
for lit in est_inicial:
  lit = '('+re.sub('\(|, ', ' ', lit)
  if 'not_' in lit:
    init += '\n\t (not (' + lit.split('not_')[1] + ')'
  else:
    init += '\n' + lit
init += '\n\n; action literals'
for a in acoes_prob:
  a = re.sub('\(|, ', ' ', a)
  init += '\n(' + a
init += '\n)'

goal = '(:goal (and'
for lit in est_meta:
  lit = '('+re.sub('\(', ' ', lit)
  if 'not_' in lit:
    goal += '\n\t (not (' + lit.split('not_')[1] + ')'
  else:
    goal += '\n\t ' + lit
goal += '\n\t)\n)'

problema = '(define (problem '+ nome + ''') 
    (:domain '''+ nome + ')\n' + obj_str + '\n' + init + '\n' + goal + '\n)'

if print_problem: print(problema)

# Resolução do problema

In [60]:
custos = []
for a in A:
  if 'castarmagia' in str(a) or 'socardenovo' in str(a): custos.append(3)
  elif 'prepararmagia' in str(a): custos.append(2)
  else: custos.append(1)

In [61]:
#@title Resolver

# Cadastro do env
import re # parsing do nome
nome = re.findall('(?<=\(define \(domain )\w+', dominio)[0]

# criação de arquivos
domain_file = open(nome+".pddl", "w")
domain_file.write(dominio)
domain_file.close()

problem_file = open('problem0.pddl', 'w')
problem_file.write(problema)
problem_file.close()

# movendo para a pasta do pddlgym
import os
try: os.mkdir('/usr/local/lib/python3.7/dist-packages/pddlgym/pddl/'+str(nome))
except: pass

import shutil
try: shutil.move(os.getcwd()+'/'+nome+'.pddl', '/usr/local/lib/python3.7/dist-packages/pddlgym/pddl/'+nome+'.pddl')
except: pass

try: shutil.move(os.getcwd()+'/problem0.pddl', '/usr/local/lib/python3.7/dist-packages/pddlgym/pddl/'+nome+'/problem0.pddl')
except: pass

import pddlgym
try: pddlgym.register_pddl_env(nome, False, {"raise_error_on_invalid_action": False})
except: pass
# fim cadastro do env


import numpy as np
from pddlgym.core import get_successor_states, InvalidAction
from pddlgym.inference import check_goal
import argparse
import os
import sys
import gym
import imageio
import pddlgym
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime
from pddlgym.inference import check_goal

matplotlib.use('agg')

sys.setrecursionlimit(5000)

def run_episode(pi,
                env,
                n_steps=100,
                print_history=False):
    obs, _ = env.reset()

    cum_reward = 0
    for i in range(1, n_steps + 1):
        old_obs = obs
        obs, reward, done, _ = env.step(pi(obs))
        cum_reward += reward
        if print_history:
            print(pi(old_obs))

        if done:
            break
    return i, cum_reward


def get_all_reachable(s, A, env, reach=None):
    reach = {} if not reach else reach

    reach[s] = {}
    for a in A:
        try:
            succ = get_successor_states(s,
                                        a,
                                        env.domain,
                                        raise_error_on_invalid_action=True,
                                        return_probs=True)
        except InvalidAction:
            succ = {s: 1.0}
        reach[s][a] = {s_: prob for s_, prob in succ.items()}
        for s_ in succ:
            if s_ not in reach:
                reach.update(get_all_reachable(s_, A, env, reach))
    return reach


def vi(S, succ_states, A, V_i, G_i, goal, env, gamma, epsilon):

    V = np.zeros(len(V_i))
    pi = np.full(len(V_i), None)

    contador = 0

    i = 0
    diff = np.inf
    while True:
        print('Iteration', i, diff)
        V_ = np.copy(V)

        for s in S:
            if check_goal(s, goal):
                continue
            Q = np.zeros(len(A))
            for i_a, a in enumerate(A):
                cost = custos[i_a]
                succ = succ_states[s, a]

                probs = np.fromiter(iter(succ.values()), dtype=float)
                succ_i = [V_i[succ_s] for succ_s in succ_states[s, a]]
                Q[i_a] = cost + np.dot(probs, gamma * V_[succ_i])

            V[V_i[s]] = np.min(Q)
            pi[V_i[s]] = A[np.argmin(Q)]
            contador += 1

        diff = np.linalg.norm(V_ - V, np.inf)
        if diff < epsilon:
            break
        i += 1
    return V, pi, contador

env = gym.make('PDDLEnv'+nome.capitalize()+'-v0')
problem_index = 0
env.fix_problem_index(problem_index)
problem = env.problems[problem_index]
goal = problem.goal
prob_objects = frozenset(problem.objects)

obs, _ = env.reset()
A = list(sorted(env.action_space.all_ground_literals(obs, valid_only=False)))

reach = get_all_reachable(obs, A, env)
S = list(sorted([s for s in reach]))

V_i = {s: i for i, s in enumerate(S)}
G_i = [V_i[s] for s in V_i if check_goal(s, goal)]

succ_states = {s: {} for s in reach}
for s in reach:
    for a in A:
        succ_states[s, a] = reach[s][a]
contador = 0
V, pi, contador = vi(S, succ_states, A, V_i, G_i, goal, env, 0.999, 0.01)
pi_func = lambda s: pi[V_i[s]]

n_episodes = 1000

print('Resposta:')
print('-----------------------------')
_, goal = run_episode(pi_func,
                      env,
                      n_steps=50,
                      print_history=True)
print('-----------------------------')

Iteration 0 inf
Iteration 1 1.0
Iteration 2 0.9990000000000001
Iteration 3 0.9980009999999999
Iteration 4 0.9970029989999998
Iteration 5 0.9960059960009997
Iteration 6 0.9950099900049993
Iteration 7 0.9940149800149936
Iteration 8 0.9930209650349786
Iteration 9 0.9920279440699442
Iteration 10 0.9910359161258739
Iteration 11 0.9900448802097479
Iteration 12 0.9890548353295383
Iteration 13 0.9880657804942086
Iteration 14 0.9870777147137151
Iteration 15 0.9860906369990001
Iteration 16 0.9851045463620025
Iteration 17 0.9841194418156398
Iteration 18 0.9831353223738226
Iteration 19 0.9821521870514509
Iteration 20 0.9811700348643981
Iteration 21 0.980188864829536
Iteration 22 0.9792086759647063
Iteration 23 0.9782294672887417
Iteration 24 0.9772512378214522
Iteration 25 0.9762739865836316
Iteration 26 0.9752977125970475
Iteration 27 0.9743224148844511
Iteration 28 0.9733480924695641
Iteration 29 0.9723747443770954
Iteration 30 0.9714023696327168
Iteration 31 0.9704309672630842
Iteration 32 0.96

In [56]:
#@title Joguinho
def print_acao(a):
  a = str(a)
  print('->', a.split('(')[1].split(',')[0].split('_')[1].split(':')[0], 'usa', a.split('(')[0], 'em', a.split('(')[1].split(',')[1].split('_')[1].split(':')[0])

def print_estado(s):
  for i in s.literals:
    i = str(i)
    print(i)

def testa_meta(s):
  l = str(s.literals)
  if 'Notvivo' in l:
    print(l.split('Notvivo')[1])
    return True
  return False

A_usuario = [A[i] for i in [3, 5, 9, 11, 13]]

print('Você é o vilão.')
print('----------------------------------\n')

estado_atual = S[0]
while True:
  aplicaveis = []
  for i_a, a in enumerate(A_usuario):
    try:
      get_successor_states(estado_atual, a, env.domain, raise_error_on_invalid_action=True)
      print(i_a, '-', str(a).split('(')[0])
      aplicaveis.append(a)
    except: continue
  try:
    acao_user = A_usuario[int(input('Sua ação: '))]
    if acao_user not in aplicaveis: raise Exception('acao invalida')
    print_acao(acao_user)
    estado_atual = list(succ_states[estado_atual, acao_user].keys())[0]
    if 'nocautear' in str(acao_user):
      print('Você venceu!')
      break
  except: 
    print('Ação inválida.')
    break

  acao_IA = pi[S.index(list(succ_states[estado_atual, acao_user].keys())[0])]
  print_acao(acao_IA)
  if 'nocautear' in str(acao_IA): 
    print('Você perdeu!')
    break
  estado_atual = list(succ_states[estado_atual, acao_IA].keys())[0]
  print()

Você é o vilão.
----------------------------------

2 - prepararmagia
3 - socar
Sua ação: 3
-> vilao usa socar em heroi
-> heroi usa prepararmagia em magiaheroi

2 - prepararmagia
3 - socar
4 - socardenovo
Sua ação: 4
-> vilao usa socardenovo em heroi
-> heroi usa castarmagia em vilao

1 - nocautear
2 - prepararmagia
3 - socar
4 - socardenovo
Sua ação: 1
-> vilao usa nocautear em heroi
Você venceu!
